# Introduction to Data Science
## Homework 2

    Student Name: Gina Holden

    Student Netid: gh1407
***

### Part 1: Case study (5 Points)
- Read [this article](http://www.nytimes.com/2012/02/19/magazine/shopping-habits.html) in the New York Times.
- Use what we've learned in class and from the book to describe how one could set Target's problem up as a predictive modeling problem, such that they could have gotten the results that they did.  Formulate your solution as a proposed plan using our data science terminology.  Include all the aspects of the formulation that you see as relevant to solving the problem.  Be precise but concise.

#### Data Set formation
The article mentions Target has access to a baby shower registry. Since we are sure these people are pregnant and furthermore we know what trimester they are in, we can use the past purchase data associated with their guest id to create a labeled dataset. The primary key for each observation would be the guest id, and the features could be certain product purchases. For example, "has bought unscented lotion in past week" or "has bought extra large bag of cotton balls in past week." The values for the columns could be booleans. Alternatively, you could count the number of purchases so the columns could be "number of purchases of unscented lotion". The target value for each observation could be boolean: "is pregnant" or you could codify the trimesters and pregnant vs not pregnant into values like 0-4 and try to predict not only if they are pregnant but what trimester they are in. 
Once you have all your positive cases from the registry and past purchase log, we need to find out the negative observations, women who are not pregnant, to train the model against. There is not a definite way to determine if a woman is not pregnant, but we can take a guess. You could take all the women (with a similar demographic as your positive data set, ie no elderly women or children) who have not bought anything remotely baby related within the past six months. Then you analyze their past spending and calculate the same feature vector you collected for the pregnant women. 
Once you have all that data, you could hold out some of it from training to test with later. 
#### Model Selection
Since we have only learned about binary regression in class, I would simplify the problem to just predict whether the person is pregnant or not. Depending on the outcome of initial data exploration, logistic regression may be a good choice. you will have to explore the variables and make sure there are no variables that are highly correlated with eachother, and maybe throw out any variables that don't show significant correlation with the target. 
#### Model fitting
Is easy once you have the features set up you just use LogisticRegression from scikit learn. before you feed your data into the algorithm you could split out the test data using train_test_split method. once you have that, it's easy to score with the .score function. 

### Part 2: Exploring data in the command line (4 Points - 1 each)
For this part we will be using the data file located in `"advertising_events.csv"`. This file consists of records that pertain to some online advertising events on a given day. There are 4 comma separated columns in this order: `userid`, `timestamp`, `domain`, and `action`. These fields are of type `int`, `int`, `string`, and `int` respectively. Answer the following questions using Linux/Unix bash commands. All questions can be answered in one line (sometimes, with pipes)! Some questions will have many possible solutions. Don't forget that in IPython notebooks you must prefix all bash commands with an exclamation point, i.e. `"!command arguments"`.

[Hints: You can experiment with whatever you want in the notebook and then delete things to construct your answer later.  You can also use ssh to use the actual bash shell in your terminal and then just paste your answers here. Recall that once you enter the "!" then filename completion should work. Also, these are standard data exploration commands that are quick and easy to use in a terminal or in the notebook. We don't cover command line operations formally in this class, but these are worth learning (and thus are part of the HW). Be resourceful. Use whatever online cheat sheets or Stackoverflow to answer the question.]

1\. How many records (lines) are in this file (look up wc)?

In [18]:
! wc -l advertising_events.csv

   10341 advertising_events.csv


2\. How many unique users are in this file? (hint: consider the 'cut' command and use pipe operator '|')

In [19]:
! cut -d "," -f 1 advertising_events.csv  | sort -u | wc -l

     732


3\. Rank all domains by the number of visits they received in descending order. (hint: consider the 'cut', 'uniq' and 'sort' commands and the pipe operator).

In [20]:
! cut -d "," -f 3 advertising_events.csv  | sort | uniq -c  | sort -r

3114 google.com
2092 facebook.com
1036 youtube.com
1034 yahoo.com
1022 baidu.com
 513 wikipedia.org
 511 amazon.com
 382 qq.com
 321 twitter.com
 316 taobao.com


4\. List all records for the user with user id 37. (hint: this can be done using 'grep')

In [21]:
! awk -F, '$1 == 37' advertising_events.csv

37,648061658,google.com,0
37,642479972,google.com,2
37,644493341,facebook.com,2
37,654941318,facebook.com,1
37,649979874,baidu.com,1
37,653061949,yahoo.com,1
37,655020469,google.com,3
37,640878012,amazon.com,0
37,659864136,youtube.com,1
37,640361378,yahoo.com,1
37,653862134,facebook.com,0
37,648828970,youtube.com,0


### Part 3: Dealing with data Pythonically (16 Points)

In [22]:
# You might find these packages useful. You may import any others you want!
import pandas as pd
import numpy as np

1\. Load the data set `"ads_dataset.tsv"` into a Python Pandas data frame called `ads`.

In [23]:
ads = pd.read_csv("ads_dataset.tsv", sep='\t', header=(0))
ads = ads.reset_index(drop=True)
ads

isbuyer  buy_freq  visit_freq  buy_interval  sv_interval  \
0            0       NaN           1           0.0     0.000000   
1            0       NaN           1           0.0     0.000000   
2            0       NaN           1           0.0     0.000000   
3            0       NaN           1           0.0     0.000000   
4            0       NaN           2           0.0     0.500000   
...        ...       ...         ...           ...          ...   
54579        0       NaN           3           0.0    30.979170   
54580        0       NaN           2           0.0     1.041667   
54581        0       NaN           1           0.0     0.000000   
54582        0       NaN           1           0.0     0.000000   
54583        0       NaN           1           0.0     0.000000   

       expected_time_buy  expected_time_visit  last_buy  last_visit  \
0                    0.0             0.000000       106         106   
1                    0.0             0.000000        72          72   
2                    0.0             0.000000         5           5   
3                    0.0             0.000000         6           6   
4                    0.0          -101.149300       101         101   
...                  ...                  ...       ...         ...   
54579                0.0            12.621240         8           8   
54580                0.0            -0.916713         1           1   
54581                0.0             0.000000        20          20   
54582                0.0             0.000000       180         180   
54583                0.0             0.000000       185         185   

       multiple_buy  multiple_visit  uniq_urls  num_checkins  y_buy  
0                 0               0        169          2130      0  
1                 0               0        154          1100      0  
2                 0               0          4            12      0  
3                 0               0        150           539      0  
4                 0               1        103           362      0  
...             ...             ...        ...           ...    ...  
54579             0               1        168          2080      0  
54580             0               1          1            15      0  
54581             0               0        132           556      0  
54582             0               0         71           400      0  
54583             0               0         77           401      0  

[54584 rows x 14 columns]

2\. Write a Python function called `getDfSummary()` that does the following:
- Takes as input a data frame
- For each variable in the data frame calculates the following features:
  - `number_nan` to count the number of missing not-a-number values
  - Ignoring missing, NA, and Null values:
    - `number_distinct` to count the number of distinct values a variable can take on
    - `mean`, `max`, `min`, `std` (standard deviation), and `25%`, `50%`, `75%` to correspond to the appropriate percentiles
- All of these new features should be loaded in a new data frame. Each row of the data frame should be a variable from the input data frame, and the columns should be the new summary features.
- Returns this new data frame containing all of the summary information

Hint: The pandas `describe()` [(manual page)](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.describe.html) method returns a useful series of values that can be used here.

In [26]:
def getDfSummary(input_data):
    num_uniq = input_data.nunique().values
    num_rows = len(input_data)
    print(input_data.isna().sum())
    output_data = input_data.describe().T
    output_data['number_nan'] = num_rows - output_data['count'] 
    output_data['number_distinct'] = num_uniq.T
    return output_data
getDfSummary(ads)

isbuyer                    0
buy_freq               52257
visit_freq                 0
buy_interval               0
sv_interval                0
expected_time_buy          0
expected_time_visit        0
last_buy                   0
last_visit                 0
multiple_buy               0
multiple_visit             0
uniq_urls                  0
num_checkins               0
y_buy                      0
dtype: int64


count        mean          std       min    25%    50%  \
isbuyer              54584.0    0.042632     0.202027    0.0000    0.0    0.0   
buy_freq              2327.0    1.240653     0.782228    1.0000    1.0    1.0   
visit_freq           54584.0    1.852777     2.921820    0.0000    1.0    1.0   
buy_interval         54584.0    0.210008     3.922016    0.0000    0.0    0.0   
sv_interval          54584.0    5.825610    17.595442    0.0000    0.0    0.0   
expected_time_buy    54584.0   -0.198040     4.997792 -181.9238    0.0    0.0   
expected_time_visit  54584.0  -10.210786    31.879722 -187.6156    0.0    0.0   
last_buy             54584.0   64.729335    53.476658    0.0000   18.0   51.0   
last_visit           54584.0   64.729335    53.476658    0.0000   18.0   51.0   
multiple_buy         54584.0    0.006357     0.079479    0.0000    0.0    0.0   
multiple_visit       54584.0    0.277444     0.447742    0.0000    0.0    0.0   
uniq_urls            54584.0   86.569343    61.969765   -1.0000   30.0   75.0   
num_checkins         54584.0  720.657592  1275.727306    1.0000  127.0  319.0   
y_buy                54584.0    0.004635     0.067924    0.0000    0.0    0.0   

                            75%          max  number_nan  number_distinct  
isbuyer                0.000000      1.00000         0.0                2  
buy_freq               1.000000     15.00000     52257.0               10  
visit_freq             2.000000     84.00000         0.0               64  
buy_interval           0.000000    174.62500         0.0              295  
sv_interval            0.104167    184.91670         0.0             5886  
expected_time_buy      0.000000     84.28571         0.0              348  
expected_time_visit    0.000000     91.40192         0.0            15135  
last_buy             105.000000    188.00000         0.0              189  
last_visit           105.000000    188.00000         0.0              189  
multiple_buy           0.000000      1.00000         0.0                2  
multiple_visit         1.000000      1.00000         0.0                2  
uniq_urls            155.000000    206.00000         0.0              207  
num_checkins         802.000000  37091.00000         0.0             4628  
y_buy                  0.000000      1.00000         0.0                2

3\. How long does it take for your `getDfSummary()` function to work on your `ads` data frame? Show us the results below.

Hint: `%timeit getDfSummary(ads)`

In [25]:
import timeit
timeit.timeit(lambda: getDfSummary(ads), number=100)

5.499078564000001

4\. Using the results returned from `getDfSummary()`, which fields, if any, contain missing `NaN` values?

buy_freq

5\. For the fields with missing values, does it look like the data is missing at random? Are there any other fields that correlate perfectly, or predict that the data is missing? If missing, what should the data value be?

Hint: create another data frame that has just the records with a missing value. Get a summary of this data frame using `getDfSummary()` and compare the differences. Do some feature distributions change dramatically?

buy_freq is Nan whenever isbuyer column is false. This intuitively makes sense because if they are not a buyer, how do you quantify buyer frequency? If you do want to fill in a value, you could say zero because they never buy anything. 

In [1]:
getDfSummary(ads)

NameError: name 'getDfSummary' is not defined

6\. Which variables are binary?

isbuyer, multiple_buy, uniq_urls, y_buy